In [1]:
from pandas import *  
import numpy as np
import re

import numpy as np
from scipy import stats
import scipy

from functools import reduce


In [9]:

# df_emo_all = read_excel("../../23March/情感风格相似度测试_Mar23-交付.xlsx")
# df_spk_all = read_excel("../../23March/音色相似度_Mar22/音色相似度测试_Mar22-交付.xlsx")

# df_emo_all = read_excel("./小说相似2_30Mar-交付.xlsx")
# df_spk_all_1 = read_excel("../说话人相似2_Mar30/说话人相似2_Mar30-交付.xlsx")
# df_spk_all_1 = df_spk_all_1[df_spk_all_1['group'] != '第5个人']
# df_spk_all_2 = read_excel("../../23March/音色相似度_Mar22/音色相似度测试_Mar22-交付.xlsx")
# df_spk_all = concat([df_spk_all_1, df_spk_all_2]).reset_index()

df_emo_all = read_excel("../广告相似3_30Mar/广告相似3_Mar30-交付.xlsx")
df_spk_all_1 = read_excel("../说话人相似3_Mar30/说话人相似3_Mar30-交付.xlsx")
df_spk_all_2 = read_excel("../../23March/音色相似度2_Mar22/音色相似度测试2_Mar22-交付.xlsx")
df_spk_all = concat([df_spk_all_1, df_spk_all_2]).reset_index()

In [10]:
def get_exp(x):
    res_mix = re.search(r"mix.*?generated|mrf.*?generated", x)
    if res_mix is None:
        return 'noise'
    return res_mix[0]


def get_fn(x):
    res_mix = re.search(r"Mar.*?.wav", x)
    if res_mix is None:
        return 'noise'
    return res_mix[0][:-4]

In [11]:
df_emo_all['exp'] = df_emo_all['link'].map(get_exp)
df_emo_all['fn'] = df_emo_all['link'].map(get_fn)
df_emo_all["score"] = df_emo_all["out"].map(lambda x:int(x[0]))


df_spk_all['exp'] = df_spk_all['link'].map(get_exp)
df_spk_all['fn'] = df_spk_all['link'].map(get_fn)


In [12]:
df_spk_noise = df_spk_all[df_spk_all['exp'] == 'noise']

In [13]:
df_spk_noise[df_spk_noise['out'] == 'A']

,index,link,b1,a1,b2,a2,b3,a3,b4,a4,b5,a5,text,id,group,out,exp,fn
2674,1129,https://gyzhang.oss-cn-hangzhou.aliyuncs.com/n...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,白酒经销模式颠覆行动开启，区域代理商名额抢先预约中\n,222,第7个人,A,noise,noise


In [14]:
exp_list = list(set(df_spk_all['exp'].values.tolist()))

In [15]:

for num, exp_name in enumerate(exp_list):
    print(exp_name)
    df_emo = df_emo_all[df_emo_all['exp'] == exp_name]
    df_spk = df_spk_all[df_spk_all['exp'] == exp_name]

    
    s_df = df_emo['score']

    m = np.mean(s_df) 
    se = scipy.stats.sem(s_df)

    print(f"mean ${round(m, 2)} \pm {round(scipy.stats.sem(s_df * scipy.stats.t.ppf((1 + 0.95) / 2., len(s_df)-1))/1.8, 2)}$")

    print("speaker similarity") 
    print(f"Source speaker ${round(sum(df_spk['out'] == 'A')/len(df_spk)*100,2)}\%$")
    print(f"Target speaker  ${round(sum(df_spk['out'] == 'B')/len(df_spk)*100,2)}\%$")
    print(f"Undetermined ${round(sum(df_spk['out'] == '无法确定')/len(df_spk)*100, 2)}\%$")
    df_spk_new = df_spk[df_spk['out'] != '无法确定']['out']
    df_A1 = (df_spk_new == 'A')*1
    df_B1 = (df_spk_new == 'B')*1
    zscore, prob = stats.ttest_ind(df_A1, df_B1, equal_var=False)
    print("p value", prob)

mix6_pt%2Fmix6_ad_test_generated
mean $3.5 \pm 0.05$
speaker similarity
Source speaker $5.0\%$
Target speaker  $95.0\%$
Undetermined $0.0\%$
p value 8.059766552635702e-218
mix6_bert_emoid%2Fmix6_ad_topline_generated
mean $4.69 \pm 0.04$
speaker similarity
Source speaker $98.67\%$
Target speaker  $1.33\%$
Undetermined $0.0\%$
p value 0.0
mix6_semip_bert%2Fmix6_test_ad_generated
mean $3.59 \pm 0.06$
speaker similarity
Source speaker $7.33\%$
Target speaker  $92.33\%$
Undetermined $0.33\%$
p value 2.499039870290186e-170
mix5_semip%2Fmix5_test_ad_generated
mean $3.52 \pm 0.06$
speaker similarity
Source speaker $8.0\%$
Target speaker  $92.0\%$
Undetermined $0.0\%$
p value 6.3274836799707095e-161
mix6_semip_bert_auto2%2Fmix6_test_ad_generated
mean $3.44 \pm 0.06$
speaker similarity
Source speaker $8.67\%$
Target speaker  $91.33\%$
Undetermined $0.0\%$
p value 1.8097524536123358e-151
mix6_stokens_n10h4_nope%2Fmix6_ad_test_generated
mean $3.44 \pm 0.05$
speaker similarity
Source speaker $3.0\%

/Users/guangyanzhang/miniconda3/envs/aligner/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3723: RuntimeWarning: Degrees of freedom <= 0 for slice
  return _methods._var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/Users/guangyanzhang/miniconda3/envs/aligner/lib/python3.8/site-packages/numpy/core/_methods.py:254: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [16]:

for num, exp_name in enumerate(exp_list):
    print(exp_name)
    df_emo = df_emo_all[df_emo_all['exp'] == exp_name]
    df_spk = df_spk_all[df_spk_all['exp'] == exp_name]

    
    s_df = df_emo['score']

    m = np.mean(s_df) 
    se = scipy.stats.sem(s_df)

    print(f"mean ${round(m, 2)} \pm {round(scipy.stats.sem(s_df * scipy.stats.t.ppf((1 + 0.95) / 2., len(s_df)-1))/1.8, 2)}$")

    print("speaker similarity") 
    
    print("Target speaker", sum(df_spk['out'] == 'B')/len(df_spk))
    print("Source speaker", sum(df_spk['out'] == 'A')/len(df_spk))
    print("Undetermined", sum(df_spk['out'] == '无法确定')/len(df_spk))
    df_spk_new = df_spk[df_spk['out'] != '无法确定']['out']
    df_A1 = (df_spk_new == 'A')*1
    df_B1 = (df_spk_new == 'B')*1
    zscore, prob = stats.ttest_ind(df_A1, df_B1, equal_var=False)
    print("p value", prob)

mix6_pt%2Fmix6_ad_test_generated
mean $3.5 \pm 0.05$
speaker similarity
Target speaker 0.95
Source speaker 0.05
Undetermined 0.0
p value 8.059766552635702e-218
mix6_bert_emoid%2Fmix6_ad_topline_generated
mean $4.69 \pm 0.04$
speaker similarity
Target speaker 0.013333333333333334
Source speaker 0.9866666666666667
Undetermined 0.0
p value 0.0
mix6_semip_bert%2Fmix6_test_ad_generated
mean $3.59 \pm 0.06$
speaker similarity
Target speaker 0.9233333333333333
Source speaker 0.07333333333333333
Undetermined 0.0033333333333333335
p value 2.499039870290186e-170
mix5_semip%2Fmix5_test_ad_generated
mean $3.52 \pm 0.06$
speaker similarity
Target speaker 0.92
Source speaker 0.08
Undetermined 0.0
p value 6.3274836799707095e-161
mix6_semip_bert_auto2%2Fmix6_test_ad_generated
mean $3.44 \pm 0.06$
speaker similarity
Target speaker 0.9133333333333333
Source speaker 0.08666666666666667
Undetermined 0.0
p value 1.8097524536123358e-151
mix6_stokens_n10h4_nope%2Fmix6_ad_test_generated
mean $3.44 \pm 0.05$
s

In [ ]:
score_list1 = []

for num, exp_name in enumerate(exp_list):
    df_emo = df_emo_all[df_emo_all['exp'] == exp_name]    
    df_emo_score = df_emo.groupby('fn', as_index=False).mean().drop(columns=['id']).rename(columns={"score": "emo" + '_' + str(num)})
    score_list1.append(df_emo_score)

In [ ]:
df_merged = reduce(lambda  left,right: merge(left,right,on=['fn'],
                                            how='outer'), score_list1)

In [ ]:
df_merged.sort_values(by=['emo_3'])

In [ ]:
for emo in emo_list:
    new_df = df[df['emo'] == emo]
    new_df = new_df[new_df["score"] == 1]
    print(len(new_df.index.values))
    del_index = new_df.index.values
    df = df.drop(del_index)